In [21]:
from crewai import Agent, LLM

llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)


In [34]:
from typing import List
from pydantic import BaseModel, Field

class topGainersList(BaseModel):
    topGainersList: List[str] = Field(..., description="List of top gainers stock options")

In [35]:
from crewai_tools import tool
import yfinance as yf
import pandas as pd
import requests



def getName(t):
    try:
        t = t.replace("+","")
        company = yf.Ticker(t)
        company_name = company.info['longName']
        return company_name
    except:
        return "Unknow"
        
@tool("topGainersTool")
def topGainersTool() -> list:
    """Gives the list top gainers of the stock options. """
    
    # Function logic here
    url = 'https://www.alphavantage.co/query?function=TOP_GAINERS_LOSERS&apikey=KIC1EWE0NFRH4Z8N'
    r = requests.get(url)
    data = r.json()
    
    df = pd.DataFrame(data['top_gainers'])
    df['Company Name'] = df.ticker.apply(getName)
    return list(df['Company Name'][:5].values)


In [36]:
topGainersAgent = Agent(
    role='Get the top gainers stock options',
    goal='Getting the top gainers stock options',
    backstory="An Stock Analyst AI assistant extracting top performers or top gainers of the stock from the top gainers tool.",
    llm=llm,
    tools=[topGainersTool]
)

In [37]:
from crewai import Task
topGainersTask = Task(
    description='Extracting the top gainers stock options',
    agent=topGainersAgent,
    expected_output='A list of top gainers stock options',
    tools=[topGainersTool],
)

In [38]:
from crewai import Agent, Task, Crew

crew = Crew(
  agents=[
    topGainersAgent
  ],
  tasks=[
    topGainersTask
  ],
  verbose=True
)

2024-11-03 01:01:20,369 - 4752 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [39]:
result = crew.kickoff()

# Agent: Get the top gainers stock options
## Task: Extracting the top gainers stock options


# Agent: Get the top gainers stock options
## Thought: Action: {"topGainersTool": {}}
## Using tool: {"topGainersTool": {}}
## Tool Input: 
""
## Tool Output: 
I encountered an error: Action '{"topGainersTool": {}}' don't exist, these are the only available Actions:
Tool Name: topGainersTool
Tool Description: topGainersTool() - Gives the list top gainers of the stock options.  
Tool Arguments: {}
Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same time. To Use the following format:

Thought: you should always think about what to do
Action: the action to take, should be one of [topGainersTool]
Action Input: the input to the action, dictionary enclosed in curly braces
Observation: the result of the action
... (this Thought/Action/Action Input/Result can repeat N times)
Thought: I now can give a great answer
Final Answer: Your final answer